In [9]:
import drms
from sunpy.util.metadata import MetaDict
import sunpy.io.fits
from sunpy.map import Map

# Using `drms` to Retrieve AIA Images

If you're in the fortunate position of not actually having to *download* the data and only need the file paths and keywords, then it may be easier to use the `drms` module directly.

However, because these files are not actually exported by JSOC, the image and header data are not recombined into a single file. This means we have to do this ourselves.

First, create a `drms` client.

In [2]:
client = drms.Client()

Next, construct a `drms` query for the same dataset in the main tutorial.

In [26]:
keys,paths = client.query(
    'aia.lev1_euv_12s[2010-09-29T23:51:36/13h@4h][171]',
    key=drms.const.all,
    seg='image',
)

The `paths` variable gives us a table with a single column that gives the "SUMS" paths to the FITS file for each image.

In [27]:
paths

,image
0,/SUM83/D179162188/S00000/image_lev1.fits
1,/SUM82/D179205869/S00000/image_lev1.fits
2,/SUM76/D179236846/S00000/image_lev1.fits
3,/SUM74/D179282080/S00000/image_lev1.fits


**NOTE:** In some cases, it may be that the headers and data have already been combined, despite not having been exported. In this case, each path in the list of files in `paths["image"]` can be passed straight to `Map` and the remaining steps are not necessary.

The `keys` variable is a table where the columns are the relevant metadata keywords and the rows are the different images.

In [28]:
keys

,T_REC,T_OBS,WAVELNTH,DATE,FSN,EXPTIME,QUALITY,ORIGIN,TELESCOP,INSTRUME,...,AGT2SVZ,AGT3SVY,AGT3SVZ,AGT4SVY,AGT4SVZ,AIMGSHEN,KEYWDDOC,LVL_NUM,T_REC_step,T_REC_epoch
0,2010-09-29T23:51:38Z,2010-09-29T23:51:49.34Z,171,2012-03-27T19:47:48Z,12898482,2.000178,0,SDO/JSOC-SDP,SDO/AIA,AIA_3,...,-7,-1,-2,0,-4,13,https://www.lmsal.com/sdodocs/doc?cmd=dcur&pro...,1.0,12.0,1993.01.01_00:00:04_TAI
1,2010-09-30T03:51:38Z,2010-09-30T03:51:49.34Z,171,2012-03-27T19:48:08Z,12908082,2.000187,0,SDO/JSOC-SDP,SDO/AIA,AIA_3,...,-3,0,2,1,0,13,https://www.lmsal.com/sdodocs/doc?cmd=dcur&pro...,1.0,12.0,1993.01.01_00:00:04_TAI
2,2010-09-30T07:51:38Z,2010-09-30T07:51:49.34Z,171,2012-03-27T19:48:27Z,12916896,1.999605,0,SDO/JSOC-SDP,SDO/AIA,AIA_3,...,9,2,1,2,-4,4,https://www.lmsal.com/sdodocs/doc?cmd=dcur&pro...,1.0,12.0,1993.01.01_00:00:04_TAI
3,2010-09-30T11:51:38Z,2010-09-30T11:51:49.34Z,171,2012-03-27T19:48:46Z,12926496,1.999636,0,SDO/JSOC-SDP,SDO/AIA,AIA_3,...,1,-1,1,2,-3,4,https://www.lmsal.com/sdodocs/doc?cmd=dcur&pro...,1.0,12.0,1993.01.01_00:00:04_TAI


For each row in the `keys` table, we'll create a metadata object representing the header of each image.

In [23]:
headers = [MetaDict({k: r[k] for k in keys.columns})
           for _,r in keys.iterrows()]

Then, we'll use the `sunpy.io.fits` to read each of the FITS files. Note that for this to work, these have to be valid filepaths on your local machine.

In [ ]:
data = [sunpy.io.fits.read(p)[1][0] for p in paths['image']]

Lastly, we'll combine the data and headers into a single `Map` object.

In [ ]:
maps = [Map(d,h) for d,h in zip(data,headers)]

This workflow will work for any images available via DRMS/JSOC that have not been combined with their headers.